In [8]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import numpy as np

scaler = StandardScaler()
dataset = pd.read_csv("Benign_train.pcap.csv")
scaled_dataset = scaler.fit_transform(dataset)
model = IsolationForest(n_estimators=50,contamination="auto")
dataset = dataset.astype(float)

model.fit(scaled_dataset)

def simulate_flow(base_index, dataset, scaler, model, steps=15):
    base_flow = dataset.iloc[[base_index]].copy()
    stacked_flows = []

    flow = base_flow.copy()

    for i in range(steps):
        flow = flow.copy()

        flow.loc[:, "syn_count"] += 2 * (i + 1)
        flow.loc[:, "rst_count"] += 1.0 * (i + 1)
        flow.loc[:, "fin_count"] += 0.8 * (i + 1)
        flow.loc[:, "ack_count"] *= (0.95 ** (i + 1))

        stacked_flows.append(flow)

    stacked_df = pd.concat(stacked_flows, ignore_index=True)
    stacked_scaled = scaler.transform(stacked_df)

    predictions = model.predict(stacked_scaled)
    scores = model.decision_function(stacked_scaled)

    return predictions, scores

rows = [5, 18, 35, 57, 74, 92, 110, 145, 180, 215]

for idx in rows:
    print(f"\n==== Base flow iloc[{idx}] ====")
    preds, scores = simulate_flow(idx, dataset, scaler, model)

    for i, (p, s) in enumerate(zip(preds, scores)):
        print(f"Window {i+1}: pred= {p} , score= {s:.4f}")


==== Base flow iloc[5] ====
Window 1: pred= 1 , score= 0.0934
Window 2: pred= 1 , score= 0.0227
Window 3: pred= 1 , score= 0.0106
Window 4: pred= 1 , score= 0.0042
Window 5: pred= -1 , score= -0.0073
Window 6: pred= -1 , score= -0.0187
Window 7: pred= -1 , score= -0.0187
Window 8: pred= -1 , score= -0.0187
Window 9: pred= -1 , score= -0.0187
Window 10: pred= -1 , score= -0.0187
Window 11: pred= -1 , score= -0.0223
Window 12: pred= -1 , score= -0.0223
Window 13: pred= -1 , score= -0.0223
Window 14: pred= -1 , score= -0.0223
Window 15: pred= -1 , score= -0.0223

==== Base flow iloc[18] ====
Window 1: pred= 1 , score= 0.1100
Window 2: pred= 1 , score= 0.0334
Window 3: pred= 1 , score= 0.0249
Window 4: pred= 1 , score= 0.0187
Window 5: pred= 1 , score= 0.0133
Window 6: pred= 1 , score= 0.0007
Window 7: pred= 1 , score= 0.0007
Window 8: pred= 1 , score= 0.0007
Window 9: pred= 1 , score= 0.0007
Window 10: pred= 1 , score= 0.0007
Window 11: pred= -1 , score= -0.0027
Window 12: pred= -1 , sco